## Get inflow Ux for all WF simulations

In [1]:
# Built-in modules
import csv
import json
import multiprocessing
import os
import time

# Third-party modules
import numpy as np
from scipy.io import loadmat
from matplotlib import cm, pyplot as plt
from scipy.io import savemat
import yaml

# Local modules
from src.data.caffa3dMBRi_gz_21_0009 import caffa3dMBRi_gz
from src.data.make_dataset import get_image_from_simulation

# Load config file
with open('config.yaml') as file:
    config = yaml.safe_load(file)

# Load turns
with open(os.path.join('data','aux', 'metadata','turns.json')) as f:
    turns = json.load(f)
# Load selected simulation time steps for temporal window in order to calculate UMean
with open(os.path.join('data','aux', 'metadata','sim_steps.json')) as f:
    t = json.load(f)

In [2]:
inflow_ux = {}
inflow_uy = {}

# iterate over precursors
for prec in config['data']['precs']:
    # iterate over angles
    for key, value in turns.items():
        # define case name
        case = f"n{prec}{key}"
        # iterate over Ux and Uy
        for comp, comp_names in enumerate(['Ux', 'Uy']):
            image, _, grid_x , grid_y = get_image_from_simulation(
                case,
                config['data']['z'],
                t[prec],
                'UMean',
                comp
                )
            # get inflow wind profile
            if comp_names == 'Ux':
                inflow_ux[case] = image[:, 0]
            elif comp_names == 'Uy':
                inflow_uy[case] = image[:, 0]
        break
    break

In [9]:
image[0,2] - image[0, 1]

0.0020339713490276773

In [16]:
coord_layout = loadmat(
    os.path.join('data', 'aux', 'metadata','coord_layout.mat')
)

In [34]:
coord_layout['coord_layout'][:, 4, :]

array([[270., 270., 270., 270., 270., 270., 270.],
       [270., 270., 270., 270., 270., 270., 270.],
       [270., 270., 270., 270., 270., 270., 270.],
       [270., 270., 270., 270., 270., 270., 270.],
       [270., 270., 270., 270., 270., 270., 270.],
       [270., 270., 270., 270., 270., 270., 270.],
       [270., 270., 270., 270., 270., 270., 270.],
       [270., 270., 270., 270., 270., 270., 270.],
       [270., 270., 270., 270., 270., 270., 270.],
       [270., 270., 270., 270., 270., 270., 270.],
       [270., 270., 270., 270., 270., 270., 270.],
       [270., 270., 270., 270., 270., 270., 270.],
       [270., 270., 270., 270., 270., 270., 270.],
       [270., 270., 270., 270., 270., 270., 270.],
       [270., 270., 270., 270., 270., 270., 270.]])

In [ ]:
position_y = {'pos_y': grid_y[:, 0]}

savemat("position_y.mat", position_y)

In [37]:
position_y =loadmat('position_y_1.mat')

In [38]:
position_y['pos_y']

array([[2250.      , 2243.9517  , 2231.855   , 2219.758   , 2207.6611  ,
        2195.5645  , 2183.4678  , 2171.3708  , 2159.2742  , 2147.1772  ,
        2135.0806  , 2122.984   , 2110.8872  , 2098.7903  , 2086.6936  ,
        2074.5967  , 2062.5     , 2050.4033  , 2038.3065  , 2026.2097  ,
        2014.1128  , 2002.0161  , 1989.9193  , 1977.8225  , 1965.7257  ,
        1953.6289  , 1941.5322  , 1929.4355  , 1917.3387  , 1905.242   ,
        1893.1451  , 1881.0483  , 1868.9517  , 1856.8549  , 1844.758   ,
        1832.6613  , 1820.5645  , 1808.4678  , 1796.3711  , 1784.2743  ,
        1772.1775  , 1760.0807  , 1747.9839  , 1735.8872  , 1723.7903  ,
        1711.6935  , 1699.5967  , 1687.5     , 1675.4033  , 1663.3065  ,
        1651.2097  , 1639.1128  , 1627.0161  , 1614.9193  , 1602.8225  ,
        1590.7257  , 1578.6289  , 1566.5322  , 1554.4355  , 1542.3387  ,
        1530.242   , 1518.1451  , 1506.0483  , 1500.      , 1500.      ,
        1493.9517  , 1481.8549  , 1469.758   , 1457

In [ ]:
from scipy.io import savemat

savemat("inflow_ux.mat", inflow_ux)
savemat("inflow_uy.mat", inflow_ux)

In [ ]:
inflow_uy['n864pr']

In [ ]:
y = np.arange(0, inflow_ux[case].shape[0])

for c, inflow in zip(['x','y'], [inflow_ux, inflow_uy]):
    fig, ax = plt.subplots(1,1)

    for i, case in enumerate(inflow_uy.keys()):
        ax.plot(inflow[case], y, c=f'C{i}')
        ax.grid()


    ax.set_xlabel('$ms^{-1}$')
    ax.set_ylabel('cell points')
    ax.set_title(f'Inflow velocity ($U_{c}$)')

    fig.savefig(os.path.join('figures', f'inflow_u{c}.png'), facecolor='white', transparent=False, bbox_inches='tight', dpi=300)

In [ ]:
image[:, 0].shape